# nbdbt

> Use jupyter notebooks for dbt development

[`dbt`](https://getdbt.com) (data build tool) is revolutionizing the way we do the process of extracting value from our data. 

This package allows Jupyter notebooks to be used for developing dbt models and analyses in complement with other dbt command line tools and VS Code extensions.

## Install

`pip install nbdbt`

## How to use

The `%%dbt` cell magic allows you to create models and analyses in your dbt project.

In addition, this package also leverages the `fal` package which provides useful modules for using dbt in python.

In [ ]:
# hide
# notest
%reload_ext autoreload
%reload_ext lab_black
%autoreload 2

In [ ]:
# load dbt cell magic
%reload_ext nbdbt.dbt_cellmagic

In [ ]:
# delete my_third_model (if existing)
! rm -f ../my_dbt_project/models/my_third_model.sql

This will create a new model `my_third_model` and compile it as well

In [ ]:
%%dbt -p ../my_dbt_project -a sample1 -n notebooks/index.ipynb models/my_third_model.sql
select *
from {{ ref('my_second_dbt_model') }}

assign=sample1
project=../my_dbt_project
notebook=notebooks/index.ipynb
file=models/my_third_model.sql
06:23:17  Found 3 models, 4 tests, 0 snapshots, 3 analyses, 191 macros, 0 operations, 0 seed files, 0 sources, 0 exposures, 0 metrics
06:23:17  
06:23:18  Concurrency: 1 threads (target='dev')
06:23:18  
06:23:18  Done.


0

Leveraging the fal package, we assigned the result to the `sample1` variable

In [ ]:
sample1

RunExecutionResult(results=[RunResult(status=<RunStatus.Success: 'success'>, timing=[TimingInfo(name='compile', started_at=datetime.datetime(2022, 6, 19, 6, 23, 18, 213498), completed_at=datetime.datetime(2022, 6, 19, 6, 23, 18, 220153)), TimingInfo(name='execute', started_at=datetime.datetime(2022, 6, 19, 6, 23, 18, 220657), completed_at=datetime.datetime(2022, 6, 19, 6, 23, 18, 220692))], thread_id='Thread-5', execution_time=0.008560895919799805, adapter_response={}, message=None, failures=None, node=CompiledModelNode(raw_sql="-- AUTOGENERATED! DO NOT EDIT! File to edit: notebooks/index.ipynb (unless otherwise specified).\nselect *\nfrom {{ ref('my_second_dbt_model') }}", compiled=True, database='sample-dbt-learn-project', schema='jaffle_shop', fqn=['my_dbt_project', 'my_third_model'], unique_id='model.my_dbt_project.my_third_model', package_name='my_dbt_project', root_path='/home/butch2/play/experiments/nbdbt/my_dbt_project', path='my_third_model.sql', original_file_path='models/my_

In [ ]:
#hide
# The `ref` method allows us to run the query and save the results into a dataframe.

In [ ]:
# hide
# samples = sample1.ref(); samples # dataframe

In [ ]:
sample1

NameError: name 'sample1' is not defined